<a href="https://colab.research.google.com/github/ClouDevops101/size-limit/blob/main/recommendation-engine-clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# build a online courses recommendation system

In [ ]:
# importing libraries

In [126]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [131]:
# show available language
print(stopwords.fileids())

['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [ ]:
# Load the data 
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [133]:
#Store the data 
df = pd.read_csv('udemy-data.csv')
# show the first 3 rows of data 
df.head(3)

,Course_id,title,description,headline,category,subcategory,instructors_name,instructors_job_title,price,avg_rating,num_reviews,created
0,1070968,Ultimate Investment Banking Course,<p>Our dedicated approach and solid methodolog...,Master all the Financial Markets concepts : St...,Finance & Accounting,Finance,VBA Finance,Trainer,€94.99,4.100000,99,2017-01-11T03:06:21Z
1,1070976,"Complete E-Commerce Course - Java,Spring,Hiber...",<p><strong></strong></p> <p><strong>WELCOME T...,Step-by-step guide on how to develop a complet...,Development,Web Development,Le Deng,"ldeng.net/courses, consultant, thousands of st...",€94.99,4.708334,730,2017-01-11T03:16:09Z
2,1071018,FL Studio Sytrus Sound Design Course,<p>In this course Alex Merced will teach you a...,Making Sounds with one of the best FM Synths o...,Music,Music Production,Alex Merced,Knowledge is Freedom,Free,3.950000,327,2017-01-11T04:11:35Z


In [134]:
df.shape


(914, 12)

In [135]:
# Creat a list of important columns for the recommendation engine
columns= ['title','description','headline','category','subcategory','instructors_name','instructors_job_title']

In [136]:
# show the data 
df[columns].head(3)


,title,description,headline,category,subcategory,instructors_name,instructors_job_title
0,Ultimate Investment Banking Course,<p>Our dedicated approach and solid methodolog...,Master all the Financial Markets concepts : St...,Finance & Accounting,Finance,VBA Finance,Trainer
1,"Complete E-Commerce Course - Java,Spring,Hiber...",<p><strong></strong></p> <p><strong>WELCOME T...,Step-by-step guide on how to develop a complet...,Development,Web Development,Le Deng,"ldeng.net/courses, consultant, thousands of st..."
2,FL Studio Sytrus Sound Design Course,<p>In this course Alex Merced will teach you a...,Making Sounds with one of the best FM Synths o...,Music,Music Production,Alex Merced,Knowledge is Freedom


In [137]:
# check for missing values in the important columns
df[columns].isnull().values.any()

# data cleaning is needed

True

In [138]:
# Create a function to combine the values of the important columns into a single string
def get_important_features(data):
  important_features = []
  for i in range(0, data.shape[0]):
    important_features.append(str(data['title'][i])+' '+str(data['description'][i])+' '+str(data['headline'][i])+' '+str(data['category'][i])+' '+str(data['subcategory'][i])+' '+str(data['instructors_name'][i])+' '+str(data['instructors_job_title'][i]) )
  return important_features


In [171]:
# Create a column to hold the combined strings
df['important_features'] = get_important_features(df)

#show the data 
df.head(10)

,Course_id,title,description,headline,category,subcategory,instructors_name,instructors_job_title,price,avg_rating,num_reviews,created,important_features
0,1070968,Ultimate Investment Banking Course,<p>Our dedicated approach and solid methodolog...,Master all the Financial Markets concepts : St...,Finance & Accounting,Finance,VBA Finance,Trainer,€94.99,4.100000,99,2017-01-11T03:06:21Z,Ultimate Investment Banking Course <p>Our dedi...
1,1070976,"Complete E-Commerce Course - Java,Spring,Hiber...",<p><strong></strong></p> <p><strong>WELCOME T...,Step-by-step guide on how to develop a complet...,Development,Web Development,Le Deng,"ldeng.net/courses, consultant, thousands of st...",€94.99,4.708334,730,2017-01-11T03:16:09Z,"Complete E-Commerce Course - Java,Spring,Hiber..."
2,1071018,FL Studio Sytrus Sound Design Course,<p>In this course Alex Merced will teach you a...,Making Sounds with one of the best FM Synths o...,Music,Music Production,Alex Merced,Knowledge is Freedom,Free,3.950000,327,2017-01-11T04:11:35Z,FL Studio Sytrus Sound Design Course <p>In thi...
3,1071060,Django by Example,<p>Django is a powerful Python web framework d...,Create your own line of successful web applica...,Development,Web Development,Packt Publishing,Tech Knowledge in Motion,€94.99,4.000000,137,2017-01-11T04:44:04Z,Django by Example <p>Django is a powerful Pyth...
4,1071120,The Art & Science of Drawing / FORM & SPACE,<p>This course contains the third and fourth i...,Learn to draw in 3-Dimensions,Lifestyle,Arts & Crafts,Brent Eviston,Artist and Instructor,€44.99,4.795652,2281,2017-01-11T05:48:46Z,The Art & Science of Drawing / FORM & SPACE <p...
5,1071124,Quality Management Essentials Simplified,<h4>FREE Quality Essentials e-Course - Diversi...,Diversify your professional skill set with qua...,Business,Management,Gregory Peckford,Corporate quality/improvement/HSE management p...,Free,4.432099,3817,2017-01-11T05:49:32Z,Quality Management Essentials Simplified <h4>F...
6,1071250,Beginning C++ Game Programming - Part 1,<p>This video course is all about offering you...,Learn C++ from scratch and get started buildin...,Development,Game Development,Packt Publishing,Tech Knowledge in Motion,€94.99,3.600000,133,2017-01-11T07:28:29Z,Beginning C++ Game Programming - Part 1 <p>Thi...
7,1071416,The Amazon FBA Seller Beginner's Toolkit,<p><strong><em>If you are a beginner in the Am...,Start Your Amazon FBA Business easier and with...,Business,E-Commerce,Sorin Constantin,Online Entrepreneur And Marketer,€19.99,3.850000,17,2017-01-11T09:09:30Z,The Amazon FBA Seller Beginner's Toolkit <p><s...
8,1071470,Desenvolvedor Android Iniciante,<p><strong>Desenvolvedor Android Iniciante</st...,Aprenda conceitos fundamentais do Android para...,Development,Mobile Development,Gabriel Ferrari,Engenheiro de Software,Free,4.685714,10414,2017-01-11T10:13:07Z,Desenvolvedor Android Iniciante <p><strong>Des...
9,1071586,Learn real Arabic part 2 (package of 3 intensi...,<p>Welcome to the biggest Arabic course on Ude...,Now speak Arabic (2) Write Arabic Now (2) Arab...,Teaching & Academics,Language Learning,Mohamed Emadeldin Ibrahim Elshenawy,Teacher,€39.99,4.300000,110,2017-01-11T11:49:01Z,Learn real Arabic part 2 (package of 3 intensi...


In [140]:
# creating stop word
stop_words =  stopwords.words('arabic') + stopwords.words('azerbaijani') + stopwords.words('danish') + stopwords.words('dutch') + stopwords.words('english') + stopwords.words('finnish') + stopwords.words('french') + stopwords.words('german') + stopwords.words('greek') + stopwords.words('hungarian') + stopwords.words('indonesian') + stopwords.words('italian') + stopwords.words('kazakh') + stopwords.words('nepali') + stopwords.words('norwegian') + stopwords.words('portuguese') + stopwords.words('romanian') + stopwords.words('russian') + stopwords.words('slovene') + stopwords.words('spanish') + stopwords.words('swedish') + stopwords.words('tajik') + stopwords.words('turkish')
print(stop_words)

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [141]:
# convert the text to a matrix of token counts 
df['important_features'].replace('<[^<]+?>',' ',regex=True, inplace = True)
print (df['important_features'].head(3))
df['important_features'] = df['important_features'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
cm = CountVectorizer().fit_transform(df['important_features'])

# GEt the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
#Print the cosine similarity matrix
print(cs)


0    Ultimate Investment Banking Course  Our dedica...
1    Complete E-Commerce Course - Java,Spring,Hiber...
2    FL Studio Sytrus Sound Design Course  In this ...
Name: important_features, dtype: object
[[1.         0.14444793 0.11465759 ... 0.07872064 0.00184976 0.08834937]
 [0.14444793 1.         0.15426368 ... 0.14710146 0.07120519 0.09564078]
 [0.11465759 0.15426368 1.         ... 0.09808165 0.00460941 0.04934555]
 ...
 [0.07872064 0.14710146 0.09808165 ... 1.         0.00356027 0.0625463 ]
 [0.00184976 0.07120519 0.00460941 ... 0.00356027 1.         0.        ]
 [0.08834937 0.09564078 0.04934555 ... 0.0625463  0.         1.        ]]


In [143]:
# Get the shape of the cosine similarity matrix 
cs.shape


(914, 914)

In [181]:
# Get the title of the course that the user likes 
title_to_search = 'Dark Web Foundation: A Guide to the Deep/Dark Web 2019'

# Find the course id
course_id = df[df.title == title_to_search]['Course_id'].index
print (course_id[0])
#course_id = 251

473


In [182]:
# Create a list of enumeration for the similarity score [(course_id, similiraty score ),(..)]
scores = list(enumerate(cs[course_id[0]]))

In [183]:
#sort the list 
sorted_scores = sorted(scores, key = lambda x:x[1], reverse  = True)
sorted_scores = sorted_scores[1:]


In [184]:
#Print the sorted scores
print(sorted_scores)

[(52, 0.43150338512672964), (841, 0.4278324340747868), (136, 0.40257553661463286), (122, 0.3593425159846164), (85, 0.3569247665117696), (84, 0.356174698072259), (86, 0.3443971645541892), (855, 0.34124003353950927), (909, 0.3060272313897129), (398, 0.30591547499484667), (406, 0.30507336310048094), (484, 0.2983149031747946), (24, 0.2973812079168519), (874, 0.29584436136431785), (857, 0.2950807509354847), (144, 0.29266638036079157), (453, 0.29052376686216214), (898, 0.2899832695221925), (888, 0.28361540252781237), (48, 0.2816725079764386), (450, 0.2704294449032917), (620, 0.265408046660862), (789, 0.2651578700992434), (3, 0.2628272457389092), (614, 0.26145953179410913), (569, 0.25869382302789645), (489, 0.2556287622994522), (601, 0.2478353938838184), (153, 0.24703366090613385), (705, 0.2457760196131729), (688, 0.24034176216786118), (292, 0.23129739659608928), (698, 0.2304657548474587), (514, 0.22800286332485237), (558, 0.22522972390073215), (151, 0.2204590259862134), (384, 0.2186434444030

In [185]:
# Create a loop to print the first 7 similar movies
j = 0
print('The 7 most recommended courses : ', title_to_search, '\nare:\n')
print('ID  Course Title' )
for item in sorted_scores:
  course_title = df[df.index == item[0]]['title'].values[0]
  course_id = df[df.index == item[0]]['Course_id'].values[0]
  #course_headline = df[df.index == item[0]]['headline'].values[0]
  num_reviews = df[df.index == item[0]]['num_reviews'].values[0]
  print(course_id, course_title, " | " + str(num_reviews) )
  j=j+1
  if j>20:
    break

The 7 most recommended courses :  Dark Web Foundation: A Guide to the Deep/Dark Web 2019 
are:

ID  Course Title
1074132 Web Hacking: Become a Professional Web Pentester  | 503
1109736 Beginners Introduction to Web Development  | 224
1077852 Python: Web Development and Penetration Testing  | 13
1077472 Learn Bootstrap 4 - Make Real World Projects & Templates  | 180
1075814 XML Crash Course For Beginners  | 60
1075808 JSON Crash Course for Beginners  | 149
1075818 XML Schema (XSD) Crash Course for Beginners  | 26
1110366 Scrapy: Powerful Web Scraping & Crawling with Python  | 2489
1112294 Mastering NodeJs  | 8
1089910 Python Scrapy: Capture Dados Web de forma rápida e escalável  | 667
1090164 Aprende Cómo Crear Páginas Web Responsivas: con HTML5 y CSS3  | 27
1093608 The Most Comprehensive Web Development Course  | 475
1072248 The All-In-One Web Development Course  | 22
1111062 Create Websites from Scratch with HTML, CSS and Javascript  | 32
1110402 Build a Web Browser for Android : 2 ho